<a href="https://colab.research.google.com/github/noeramlpspta/online_retail/blob/main/User_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation Data

In [35]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from scipy import stats

In [6]:
df = pd.read_csv('/content/drive/MyDrive/Python/Online Retail Project/Online Retail Data.csv')
df

,order_id,product_code,product_name,quantity,order_date,price,customer_id
0,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.50,12346.0
1,C493411,21539,RETRO SPOTS BUTTER DISH,-1,2010-01-04 09:43:00,4.25,14590.0
2,493412,TEST001,This is a test product.,5,2010-01-04 09:53:00,4.50,12346.0
3,493413,21724,PANDA AND BUNNIES STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN
4,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1,2010-01-04 09:54:00,3.75,NaN
...,...,...,...,...,...,...,...
461768,539991,21618,4 WILDFLOWER BOTANICAL CANDLES,1,2010-12-23 16:49:00,1.25,NaN
461769,539991,72741,GRAND CHOCOLATECANDLE,4,2010-12-23 16:49:00,1.45,NaN
461770,539992,21470,FLOWER VINE RAFFIA FOOD COVER,1,2010-12-23 17:41:00,3.75,NaN
461771,539992,22258,FELT FARM ANIMAL RABBIT,1,2010-12-23 17:41:00,1.25,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461773 entries, 0 to 461772
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      461773 non-null  object 
 1   product_code  461773 non-null  object 
 2   product_name  459055 non-null  object 
 3   quantity      461773 non-null  int64  
 4   order_date    461773 non-null  object 
 5   price         461773 non-null  float64
 6   customer_id   360853 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 24.7+ MB


# Cleaning Data

In [11]:
# Membuat salinan dari df
df_retail = df.copy()
# Mengonversi kolom order_date menjadi datetime
df_retail['order_date'] = pd.to_datetime(df_retail['order_date'], errors='coerce')
# Membuat kolom year_month
df_retail['year_month'] = df_retail['order_date'].dt.to_period('M')
# Menghapus semua baris tanpa customer_id
df_retail = df_retail[~df_retail['customer_id'].isna()].copy()
# Menghapus semua baris tanpa product_name
df_retail = df_retail[~df_retail['product_name'].isna()].copy()
# Membuat semua product_name berhuruf kecil
df_retail.loc[:, 'product_name'] = df_retail['product_name'].str.lower()
# Menghapus semua baris dengan product_code atau product_name yang mengandung 'test'
df_retail = df_retail[(~df_retail['product_code'].str.lower().str.contains('test', na=False)) |
                      (~df_retail['product_name'].str.contains('test', na=False))].copy()
# Membuat kolom order_status dengan nilai 'cancelled' atau 'delivered'
df_retail.loc[:, 'order_status'] = np.where(df_retail['order_id'].str[:1] == 'C', 'cancelled', 'delivered')
# Mengubah nilai quantity yang negatif menjadi positif
df_retail.loc[:, 'quantity'] = df_retail['quantity'].abs()
# Menghapus baris dengan price bernilai negatif
df_retail = df_retail[df_retail['price'] > 0].copy()
# Membuat nilai amount sebagai perkalian antara quantity dan price
df_retail.loc[:, 'amount'] = df_retail['quantity'] * df_retail['price']
# Memperbarui product_name berdasarkan product_code yang paling sering muncul
most_freq_product_name = (df_retail.groupby(['product_code', 'product_name'], as_index=False)
                          .agg(order_cnt=('order_id', 'nunique'))
                          .sort_values(['product_code', 'order_cnt'], ascending=[True, False]))
most_freq_product_name['rank'] = most_freq_product_name.groupby('product_code')['order_cnt'].rank(method='first', ascending=False)
most_freq_product_name = most_freq_product_name[most_freq_product_name['rank'] == 1].drop(columns=['order_cnt', 'rank'])
df_retail = df_retail.merge(most_freq_product_name.rename(columns={'product_name': 'most_freq_product_name'}),
                            how='left', on='product_code')
df_retail.loc[:, 'product_name'] = df_retail['most_freq_product_name']
df_retail = df_retail.drop(columns='most_freq_product_name')
# Mengonversi customer_id menjadi string
df_retail['customer_id'] = df_retail['customer_id'].astype(str)
# Menghapus outlier
from scipy import stats
df_retail = df_retail[(np.abs(stats.zscore(df_retail[['quantity', 'amount']])) < 3).all(axis=1)]
df_retail = df_retail.reset_index(drop=True)
df_retail


,order_id,product_code,product_name,quantity,order_date,price,customer_id,year_month,order_status,amount
0,C493411,21539,red retrospot butter dish,1,2010-01-04 09:43:00,4.25,14590.0,2010-01,cancelled,4.25
1,493414,21844,red retrospot mug,36,2010-01-04 10:28:00,2.55,14590.0,2010-01,delivered,91.80
2,493414,21533,retro spot large milk jug,12,2010-01-04 10:28:00,4.25,14590.0,2010-01,delivered,51.00
3,493414,37508,new england ceramic cake server,2,2010-01-04 10:28:00,2.55,14590.0,2010-01,delivered,5.10
4,493414,35001G,hand open shape gold,2,2010-01-04 10:28:00,4.25,14590.0,2010-01,delivered,8.50
...,...,...,...,...,...,...,...,...,...,...
358464,539988,84380,set of 3 butterfly cookie cutters,1,2010-12-23 16:06:00,1.25,18116.0,2010-12,delivered,1.25
358465,539988,84849D,hot baths soap holder,1,2010-12-23 16:06:00,1.69,18116.0,2010-12,delivered,1.69
358466,539988,84849B,fairy soap soap holder,1,2010-12-23 16:06:00,1.69,18116.0,2010-12,delivered,1.69
358467,539988,22854,cream sweetheart egg holder,2,2010-12-23 16:06:00,4.95,18116.0,2010-12,delivered,9.90


In [13]:
df_retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358469 entries, 0 to 358468
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      358469 non-null  object        
 1   product_code  358469 non-null  object        
 2   product_name  358469 non-null  object        
 3   quantity      358469 non-null  int64         
 4   order_date    358469 non-null  datetime64[ns]
 5   price         358469 non-null  float64       
 6   customer_id   358469 non-null  object        
 7   year_month    358469 non-null  period[M]     
 8   order_status  358469 non-null  object        
 9   amount        358469 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(5), period[M](1)
memory usage: 27.3+ MB


# Membuat RFM Segmentation

In [20]:
# Agregasi data transaksi
df_user = df_retail.groupby('customer_id',as_index=False).agg(order_cnt=('order_id', 'nunique'),max_order_date=('order_date', 'max'),total_order_value=('amount', 'sum'))
# Mendapatkan tanggal order terbaru
today = df_retail['order_date'].max()
# Menghitung jumlah hari sejak order terakhir
df_user['day_since_last_order'] = (today - df_user['max_order_date']).dt.days
df_user

,customer_id,order_cnt,max_order_date,total_order_value,day_since_last_order
0,12346.0,5,2010-10-04 09:54:00,602.40,80
1,12608.0,1,2010-10-31 10:49:00,415.79,53
2,12745.0,2,2010-08-10 10:14:00,723.85,135
3,12746.0,2,2010-06-30 08:19:00,266.35,176
4,12747.0,19,2010-12-13 10:41:00,4094.79,10
...,...,...,...,...,...
3884,18283.0,6,2010-11-22 15:30:00,641.77,31
3885,18284.0,2,2010-10-06 12:31:00,486.68,78
3886,18285.0,1,2010-02-17 10:24:00,427.00,309
3887,18286.0,2,2010-08-20 11:57:00,941.48,125


In [21]:
df_user.describe()

,order_cnt,max_order_date,total_order_value,day_since_last_order
count,3889.000000,3889,3889.000000,3889.000000
mean,5.128568,2010-09-23 18:15:51.267678208,1544.623084,90.651581
min,1.000000,2010-01-05 12:43:00,1.250000,0.000000
25%,1.000000,2010-08-19 12:30:00,296.360000,25.000000
50%,3.000000,2010-10-26 18:45:00,648.200000,57.000000
75%,6.000000,2010-11-28 14:54:00,1585.940000,126.000000
max,163.000000,2010-12-23 16:06:00,71970.390000,352.000000
std,8.499330,NaN,3434.816315,88.883201


# Buat binning dari jumlah hari sejak order terakhir yang terdiri dari 5 bins dengan batas-batasnya merupakan min, P20, P40, P60, P80, max dan beri label 1 sampai 5 dari bin tertinggi ke terendah sebagai skor recency

In [30]:
df_user['recency_score'] = pd.cut(df_user['day_since_last_order'],
                                  bins=[df_user['day_since_last_order'].min(),
                                        np.percentile(df_user['day_since_last_order'], 20),
                                        np.percentile(df_user['day_since_last_order'], 40),
                                        np.percentile(df_user['day_since_last_order'], 60),
                                        np.percentile(df_user['day_since_last_order'], 80),
                                        df_user['day_since_last_order'].max()],
                                  labels=[5, 4, 3, 2, 1],
                                  include_lowest=True).astype(int)
df_user


,customer_id,order_cnt,max_order_date,total_order_value,day_since_last_order,frequency_score,monetary_score,recency_score
0,12346.0,5,2010-10-04 09:54:00,602.40,80,4,3,2
1,12608.0,1,2010-10-31 10:49:00,415.79,53,1,2,3
2,12745.0,2,2010-08-10 10:14:00,723.85,135,2,3,2
3,12746.0,2,2010-06-30 08:19:00,266.35,176,2,2,1
4,12747.0,19,2010-12-13 10:41:00,4094.79,10,5,5,5
...,...,...,...,...,...,...,...,...
3884,18283.0,6,2010-11-22 15:30:00,641.77,31,4,3,4
3885,18284.0,2,2010-10-06 12:31:00,486.68,78,2,3,2
3886,18285.0,1,2010-02-17 10:24:00,427.00,309,1,2,1
3887,18286.0,2,2010-08-20 11:57:00,941.48,125,2,4,2


## Buat binning dari total transaksi (order) yang terdiri dari 5 bins dengan batas-batasnya merupakan min, P20, P40, P60, P80, max dan beri label 1 sampai 5 dari bin terendah ke tertinggi sebagai skor frequency

In [31]:
df_user['frequency_score'] = pd.cut(df_user['order_cnt'],
                                    bins=[0,
                                          np.percentile(df_user['order_cnt'], 20),
                                          np.percentile(df_user['order_cnt'], 40),
                                          np.percentile(df_user['order_cnt'], 60),
                                          np.percentile(df_user['order_cnt'], 80),
                                          df_user['order_cnt'].max()],
                                    labels=[1, 2, 3, 4, 5],
                                    include_lowest=True).astype(int)
df_user

,customer_id,order_cnt,max_order_date,total_order_value,day_since_last_order,frequency_score,monetary_score,recency_score
0,12346.0,5,2010-10-04 09:54:00,602.40,80,4,3,2
1,12608.0,1,2010-10-31 10:49:00,415.79,53,1,2,3
2,12745.0,2,2010-08-10 10:14:00,723.85,135,2,3,2
3,12746.0,2,2010-06-30 08:19:00,266.35,176,2,2,1
4,12747.0,19,2010-12-13 10:41:00,4094.79,10,5,5,5
...,...,...,...,...,...,...,...,...
3884,18283.0,6,2010-11-22 15:30:00,641.77,31,4,3,4
3885,18284.0,2,2010-10-06 12:31:00,486.68,78,2,3,2
3886,18285.0,1,2010-02-17 10:24:00,427.00,309,1,2,1
3887,18286.0,2,2010-08-20 11:57:00,941.48,125,2,4,2


## Buat binning dari total nilai order (order value) yang terdiri dari 5 bins dengan batas-batasnya merupakan min, P20, P40, P60, P80, max dan beri label 1 sampai 5 dari bin terendah ke tertinggi sebagai skor monetary

In [32]:
df_user['monetary_score'] = pd.cut(df_user['total_order_value'],
                                   bins=[df_user['total_order_value'].min(),
                                         np.percentile(df_user['total_order_value'], 20),
                                         np.percentile(df_user['total_order_value'], 40),
                                         np.percentile(df_user['total_order_value'], 60),
                                         np.percentile(df_user['total_order_value'], 80),
                                         df_user['total_order_value'].max()],
                                   labels=[1, 2, 3, 4, 5],
                                   include_lowest=True).astype(int)
df_user

,customer_id,order_cnt,max_order_date,total_order_value,day_since_last_order,frequency_score,monetary_score,recency_score
0,12346.0,5,2010-10-04 09:54:00,602.40,80,4,3,2
1,12608.0,1,2010-10-31 10:49:00,415.79,53,1,2,3
2,12745.0,2,2010-08-10 10:14:00,723.85,135,2,3,2
3,12746.0,2,2010-06-30 08:19:00,266.35,176,2,2,1
4,12747.0,19,2010-12-13 10:41:00,4094.79,10,5,5,5
...,...,...,...,...,...,...,...,...
3884,18283.0,6,2010-11-22 15:30:00,641.77,31,4,3,4
3885,18284.0,2,2010-10-06 12:31:00,486.68,78,2,3,2
3886,18285.0,1,2010-02-17 10:24:00,427.00,309,1,2,1
3887,18286.0,2,2010-08-20 11:57:00,941.48,125,2,4,2


## Buat kolom nama segmen berdasarkan skor recency dan frequency

In [33]:
df_user['segment'] = np.select(
    [(df_user['recency_score']==5) & (df_user['frequency_score']>=4),
     (df_user['recency_score'].between(3, 4)) & (df_user['frequency_score']>=4),
     (df_user['recency_score']>=4) & (df_user['frequency_score'].between(2, 3)),
     (df_user['recency_score']<=2) & (df_user['frequency_score']==5),
     (df_user['recency_score']==3) & (df_user['frequency_score']==3),
     (df_user['recency_score']==5) & (df_user['frequency_score']==1),
     (df_user['recency_score']==4) & (df_user['frequency_score']==1),
     (df_user['recency_score']<=2) & (df_user['frequency_score'].between(3, 4)),
     (df_user['recency_score']==3) & (df_user['frequency_score']<=2),
     (df_user['recency_score']<=2) & (df_user['frequency_score']<=2)],
    ['01-Champion', '02-Loyal Customers', '03-Potential Loyalists', "04-Can't Lose Them", '05-Need Attention',
     '06-New Customers', '07-Promising', '08-At Risk', '09-About to Sleep', '10-Hibernating']
)
df_user

,customer_id,order_cnt,max_order_date,total_order_value,day_since_last_order,frequency_score,monetary_score,recency_score,segment
0,12346.0,5,2010-10-04 09:54:00,602.40,80,4,3,2,08-At Risk
1,12608.0,1,2010-10-31 10:49:00,415.79,53,1,2,3,09-About to Sleep
2,12745.0,2,2010-08-10 10:14:00,723.85,135,2,3,2,10-Hibernating
3,12746.0,2,2010-06-30 08:19:00,266.35,176,2,2,1,10-Hibernating
4,12747.0,19,2010-12-13 10:41:00,4094.79,10,5,5,5,01-Champion
...,...,...,...,...,...,...,...,...,...
3884,18283.0,6,2010-11-22 15:30:00,641.77,31,4,3,4,02-Loyal Customers
3885,18284.0,2,2010-10-06 12:31:00,486.68,78,2,3,2,10-Hibernating
3886,18285.0,1,2010-02-17 10:24:00,427.00,309,1,2,1,10-Hibernating
3887,18286.0,2,2010-08-20 11:57:00,941.48,125,2,4,2,10-Hibernating


## Tampilkan summary dari RFM segmentation (poin 8) berupa banyaknya pengguna, rata-rata dan median dari total order, total order value, dan jumlah hari sejak order terakhir

In [34]:
summary = pd.pivot_table(df_user, index='segment',
               values=['customer_id','day_since_last_order','order_cnt','total_order_value'],
               aggfunc={'customer_id': pd.Series.nunique,
                        'day_since_last_order': [np.mean, np.median],
                        'order_cnt': [np.mean, np.median],
                        'total_order_value': [np.mean, np.median]})
summary['pct_unique'] = (summary['customer_id'] / summary['customer_id'].sum() * 100).round(1)
summary

<ipython-input-34-2376a8509f65>:1: FutureWarning: The provided callable <function mean at 0x7a84e0d99000> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  summary = pd.pivot_table(df_user, index='segment',
<ipython-input-34-2376a8509f65>:1: FutureWarning: The provided callable <function median at 0x7a84ddb20700> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  summary = pd.pivot_table(df_user, index='segment',


customer_id day_since_last_order         order_cnt  \
                           nunique                 mean median       mean   
segment                                                                     
01-Champion                    553            10.533454    9.0  15.432188   
02-Loyal Customers             549            41.200364   37.0   8.744991   
03-Potential Loyalists         514            23.083658   24.0   2.830739   
04-Can't Lose Them              62           123.274194  113.0  11.467742   
05-Need Attention              184            58.505435   59.0   3.402174   
06-New Customers                50            14.000000   16.0   1.000000   
07-Promising                   133            31.954887   32.0   1.000000   
08-At Risk                     418           141.531100  120.0   4.126794   
09-About to Sleep              370            58.175676   58.0   1.416216   
10-Hibernating                1056           197.151515  199.0   1.312500   

                              total_order_value           pct_unique  
                       median              mean    median             
segment                                                               
01-Champion              10.0       4989.208761  2773.910       14.2  
02-Loyal Customers        7.0       2618.121117  1937.050       14.1  
03-Potential Loyalists    3.0        766.076265   621.005       13.2  
04-Can't Lose Them       10.0       2851.737258  2268.405        1.6  
05-Need Attention         3.0       1004.317071   826.370        4.7  
06-New Customers          1.0        244.689000   193.675        1.3  
07-Promising              1.0        288.694135   239.460        3.4  
08-At Risk                4.0       1141.224835   866.320       10.7  
09-About to Sleep         1.0        448.176081   336.735        9.5  
10-Hibernating            1.0        342.618450   256.900       27.2